In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler


import torchvision.datasets as dset
from torchvision.transforms import v2

import matplotlib.pyplot as plt
import numpy as np
import copy

In [2]:
# download CIFAR dataset
NUM_TRAIN = 40000

transform = v2.Compose([v2.ToTensor(), v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
cifar_train = dset.CIFAR10('./datasets', train=True, download=True, transform=transform)
cifar_val = dset.CIFAR10('./datasets', train=False, download=True, transform=transform)
loader_train = DataLoader(cifar_train, batch_size=64, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))
loader_val = DataLoader(cifar_val, batch_size=64)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

c:\Users\murat\.conda\envs\edf\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified


In [3]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [4]:
def train_loop(dataloader, model, loss_fn, optimizer, reg, w_reg):
    # reg defines the type of regularization, either 'l1' or 'l2' or None
    # w_reg is the regularization weight
    model.train()
    for x, y in dataloader:
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

def eval_loop(dataloader, model, loss_fn, reg=None, w_reg=None):
    # reg defines the type of regularization, either 'l1' or 'l2' or None
    # w_reg is the regularization weight
    # modify this function the same as the train_loop
    model.eval()
    num_correct = 0
    num_samples = 0
    total_loss = 0

    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(DEVICE)
            y = y.to(DEVICE)
            pred = model(x)
            loss = loss_fn(pred, y)
            total_loss += loss.item()
            num_correct += (pred.argmax(1) == y).sum().item()
            num_samples += pred.shape[0]
    
    acc = 100*(num_correct / num_samples)
    total_loss /= len(dataloader)
    return acc, total_loss 


def run(loader_train, loader_val, model, loss_fn, optimizer, num_epochs=1, reg=None, w_reg=None):
    torch.manual_seed(0)
    np.random.seed(0)
    train_accs = []
    val_accs = []
    train_losses = []
    val_losses = []
    for i in range(num_epochs):
        train_loop(loader_train, model, loss_fn, optimizer, reg, w_reg)
        train_acc, train_loss = eval_loop(loader_train, model, loss_fn, reg, w_reg)
        val_acc, val_loss = eval_loop(loader_val, model, loss_fn, reg, w_reg)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f'Epoch {i+1}/{num_epochs}, Train Acc: {train_acc:.4f}%, Val Acc: {val_acc:.4f}%')
    print('-'*50)
    return train_accs, val_accs, train_losses, val_losses


In [7]:
# Define the model
num_epochs = 1
lr = 1e-3
weight_decay = 0
model = CustomCNN().to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [8]:
train_accs, val_accs, train_losses, val_losses = run(loader_train, loader_val, model, loss_fn, optimizer, num_epochs=num_epochs)

Epoch 1/1, Train Acc: 56.4500%, Val Acc: 55.5900%
--------------------------------------------------


In [ ]:
from torch.utils.tensorboard import SummaryWriter

model = CustomCNN()
writer = SummaryWriter("runs/CustomCNN")
dummy_input = torch.randn(1, 3, 32, 32)  # Example input size
writer.add_graph(model, dummy_input)
writer.close()


   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/5.5 MB ? eta -:--:--
   ----------- ---------------------------- 1.6/5.5 MB 5.2 MB/s eta 0:00:01
   ----------------- ---------------------- 2.4/5.5 MB 4.6 MB/s eta 0:00:01
   -------------------- ------------------- 2.9/5.5 MB 4.3 MB/s eta 0:00:01
   ------------------------ --------------- 3.4/5.5 MB 3.7 MB/s eta 0:00:01
   -------------------------- ------------- 3.7/5.5 MB 3.5 MB/s eta 0:00:01
   ---------------------------- ----------- 3.9/5.5 MB 3.3 MB/s eta 0:00:01
   -------------------------------- ------- 4.5/5.5 MB 2.9 MB/s eta 0:00:01
   -------------------------------- ------- 4.5/5.5 MB 2.9 MB/s eta 0:00:01
   ---------------------------------- ----- 4.7/5.5 MB 2.6 MB/s eta 0:00:01
   ------------------------------------ --- 5.0/5.5 MB 2.3 MB/s eta 0:00:01
   ------------------------------------ --- 5.0/5.5 MB 2.3 MB/s eta 0:00:01
   -----------------------